### Convert a list of relations defining a route to a set of coordinates

Again, we are trying to find the coordinates for public transit trip. This experiments with using osmapi directly to find the nodes that are part of a relation. Seems promising, but also seems like a lot of work to get the corner cases right. Might return to this later.

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

#### Let's see what we can do with the OSM API directly

In [ ]:
import osmapi

In [ ]:
import folium

In [ ]:
osm = osmapi.OsmApi()

In [ ]:
# Get a node
relation_details = osm.RelationGet(8094017); relation_details

In [ ]:
# Porting the perl script at https://wiki.openstreetmap.org/wiki/Relations/Relations_to_GPX
# to python
def get_way_list(relation_details):
    wl = []
    for member in relation_details["member"]:
        # print(member["ref"], member["type"])
        assert member["type"] != "relation", "This is a parent relation for child %d, expecting only child relations" % member["ref"]
        if member["type"] == "way" and member["role"] != "platform":
            wl.append(member["ref"])
    return wl

In [ ]:
wl = get_way_list(osm.RelationGet(8094017))
idx = wl.index(52869637); idx

In [ ]:
wl[74:]

In [ ]:
# way details is an array of n-1 node entries followed by a way entry
# the way entry has an "nd" field which is an array of node ids in the correct order
# the n-1 node entries are not necessarily in the correct order but provide the id -> lat,lng mapping
def get_coords_for_way(wid, prev_last_node=-1):
    lat = {}
    lon = {}
    coords_list = []
    way_details = osm.WayFull(wid)
    # print("Processing way %d with %d nodes" % (wid, len(way_details) - 1))
    for e in way_details:
        if e["type"] == "node":
            lat[e["data"]["id"]] = e["data"]["lat"]
            lon[e["data"]["id"]] = e["data"]["lon"]
        if e["type"] == "way":
            assert e["data"]["id"] == wid, "Way id mismatch! %d != %d" % (e["data"]["id"], wl[0])
            ordered_node_array = e["data"]["nd"]
            print("Comparing %d and %d" % (prev_last_node, ordered_node_array[-1]))
            if prev_last_node != -1 and ordered_node_array[-1] == prev_last_node:
                print("LAST entry %d matches prev_last_node %d, REVERSING order for %d" %
                      (ordered_node_array[-1], prev_last_node, wid))
                ordered_node_array = list(reversed(ordered_node_array))
            for on in ordered_node_array:
                coords_list.append([lon[on], lat[on]])
    return ordered_node_array, coords_list

In [ ]:
# on_list, coords_list = get_coords_for_way(27422567)
# on_list[-1], coords_list

In [ ]:
# on_list, coords_list = get_coords_for_way(367132251, 65477623)
# on_list[-1], coords_list

In [ ]:
# on_list, coords_list = get_coords_for_way(367132252, 65477686)
# on_list[-1], coords_list

In [ ]:
# on_list, coords_list = get_coords_for_way(32315631, 3710856959)
# on_list[-1], coords_list

In [ ]:
def get_coords_for_relation(rid, start_node, end_node):
    relation_details = osm.RelationGet(rid)
    wl = get_way_list(relation_details)
    print("Relation %d mapped to %d ways" % (rid, len(wl)))
    coords_list = []
    on_list = []
    prev_last_node = -1
    for wid in wl:
        w_on_list, w_coords_list = get_coords_for_way(wid, prev_last_node)
        on_list.extend(w_on_list)
        coords_list.extend(w_coords_list)
        prev_last_node = w_on_list[-1]
        print("After adding %d entries from wid %d, curr count = %d" % (len(w_on_list), wid, len(coords_list)))
    start_index = on_list.index(start_node)
    end_index = on_list.index(end_node)
    assert start_index <= end_index, "Start index %d is before end %d" % (start_index, end_index)
    return coords_list[start_index:end_index+1]

In [ ]:
coords_list = []
on_list = []
prev_last_node = -1
for wid in wl[]:
    w_on_list, w_coords_list = get_coords_for_way(wid, prev_last_node)
    on_list.extend(w_on_list)
    coords_list.extend(w_coords_list)
    prev_last_node = w_on_list[-1]
    print("After adding %d entries from wid %d, curr count = %d" % (len(w_on_list), wid, len(coords_list)))

In [ ]:
def show_coords_on_map(route_coords):
    curr_map = folium.Map()
    latlng_route_coords = [lonlat_swap(rc) for rc in route_coords]
    pl = folium.PolyLine(latlng_route_coords)
    pl.add_to(curr_map)
    for i, c in enumerate(latlng_route_coords):
        folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
    curr_map.fit_bounds(pl.get_bounds()) 
    return curr_map

In [ ]:
show_coords_on_map(coords_list)

In [ ]:
# BART from Millbrae to Richmond
# route_coords = get_coords_for_relation(2851613, 6138188702, 65409922)
# show_coords_on_map(route_coords)

In [ ]:
# Caltrain from San Jose to San Francisco
route_coords = get_coords_for_relation(8094017, 5020708467, 5465520388)

In [ ]:
# on_list.index(65409922), on_list.index(6138188702)

In [ ]:
# on_list[650:850]

In [ ]:
# [(650+i, r) for i, r in enumerate(route_coords[650:850])]

In [ ]:
# route_coords = route_coords[309:708+1]

In [ ]:
show_coords_on_map(route_coords)

In [ ]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [ ]:
# Ways that the node is part of. Note that there are two ways with different IDs (685743128 and 685743129) at different changesets
osm.NodeWays(6426793216)

In [ ]:
# NodeRelations does not work, probably because there are no relations and this is not a transitive property
osm.NodeRelations(6426793216)

In [ ]:
ways = osm.NodeWays(6426793216)
for w in ways:
    rw = osm.WayRelations(w["id"])
    print("Considering way %d" % w["id"])
    for r in rw:
        # print(r)
        print("Found relation %d with type %s" % (r["id"], r["tag"]["type"]))
    # print("Found %d relations of types %s for way %d" % (len(rw), [r["type"] for r in rw], w["id"]))
    # print("Found %d route relations for way %d" % (len([r for r in rw if r["type"] == "route"]), w["id"]))